# Plot Adsorption Isotherms

In this notebook:
1. Henry coefficient fits to experimental isotherm data
2. Experimental and Simulated adsorption isotherms for:

    (i)  Ni(PyC)2 
        (a) Xe, Kr, Ar @ T = 298 K
    (ii) Ni(PyC-m-NH2)2 
        (a) Xe, Kr, Ar @ T = 298 K 
        (b) Xe in different topologies: 
            - single-net v.s. double-net 
            - Pn v.s. P2_1/n (cis v.s. trans PyC)
3. Experimental (only) isotherms:

    (i)  Ni(PyC)2
         (a) Xe, Kr, and Ar @ T = 278 K, 288 K, 298 K 
         (b)Nitrogen adsorption and desorption @ T = 77 K
    (ii) Ni(PyC-m-NH2)2
         (a) Xe, Kr, and Ar @ T = 278 K, 288 K, and 298 K 
         (b) Nitrogen adsorption and desorption @ T = 77 K
4. DFT relaxation methods comparison plot (Xe adsorption @ T = 298 K)

In [ ]:
using PorousMaterials
using JLD2
using CSV
using DataFrames
using LinearAlgebra
using PyPlot
using PyCall
using Printf

# set plot params
PyPlot.matplotlib.style.use("ggplot")
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["font.size"] = 16;

In [ ]:
# set path to data
set_path_to_data(joinpath(pwd(), "../../data"));

**Define parameters used for locating data**

In [ ]:
###
#  properties we want to pull from simulation results dictionary
###
props_we_want = ["pressure (bar)", "fugacity (bar)", "⟨N⟩ (mmol/g)"]

###
#  Gas (adsorbate) molecules
###
adsorbates = ["Xe", "Kr", "Ar"]
molar_mass  = Dict("Xe" => 131.293, "Kr" => 83.798, "Ar" => 39.948) # [g / mol], used for unit conversions

###
#  simulation parameters
#   used to locate desired simulation files
###
pmin  = -3  # minimum pessure in log10, units: bar
pmax  = 1.0 # maximum pressure (actual value), units: bar
nstep = 25  # number of intervals in pressure range

sim_params = Dict("adsorbates"  => adsorbates,
                  "temperature" => 298.0, # units: K
                  "molecules"   => Dict(zip(adsorbates, Molecule.(adsorbates))),
                  "pressures"   => 10 .^ range(pmin, stop=log10(pmax), length=nstep),
                  "ljff"        => LJForceField("UFF"), # molecular force field model
                  "n_burn"      => 50000, # number of burn cycles in simulation
                  "n_sample"    => 50000  # number of production cycles 
                  )



In [ ]:
thing_name = 
# ("NiPyC2", "") => Dict(:marker => )

In [ ]:
adsorbate_to_color = Dict(zip(adsorbates, ["C0", "C1", "C2"]))

# adsorbate_to_marker = Dict(zip(["Xe", "Kr", "Ar"], ["v", "D", "h"]))


markers = ["o", "v", "^", ">", "<", "1", "2", "3", "4", "8", "s", "p", "P", 
            "*", "h", "H", "+", "x", "X", "D", "d", "."]


###
#  IDEA!! Make THIS A TUPLE: (Xe, sim) => "X", (Xe, exp) => "v"
data_marker = Dict(zip([1, 2, 3, 4, 5, 6],["X", "v", "P", "D", "*", "h"]))

# labels for each MOF
# mof_to_marker = Dict(zip(frameworks, ["x", "s", "P", "o", "*", ">"]))
# mof_to_color = Dict(zip(frameworks, ["C$i" for i = 1:length(frameworks)]))

# labels for each experimental data set
labels_exp = Dict("Xe" => "Xe experiment", 
                  "Kr" => "Kr experiment", 
                  "Ar" => "Ar experiment")

exp_data_to_marker = Dict(zip(["", "NH2"], ["v", "D"]))
exp_data_to_color  = Dict(zip(["", "NH2"], ["g", "b"]))

# labels for adsorbate used
labels_gas = Dict("Xe" => "Xe simulation",
                  "Kr" => "Kr simulation",
                  "Ar" => "Ar simulation")

pretty_names = Dict(("NiPyC2", "") => "Ni(PyC)" * L"_2", 
                    ("NiPyC2", "mNH2") => "Ni(PyC-" * L"m" * "-NH" * L"_2" * ")" * L"_2")